In [1]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone.core.labels import Detection
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data

import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
import numpy as np

import os
import importlib
import sys
from pathlib import Path
from datetime import datetime

import utils
import train
from train import train_one_epoch, evaluate
import transforms as T
importlib.reload(T)

here = os.getcwd()
sys.path.append(os.path.join(here, '..'))
import DatasetLoaders
importlib.reload(DatasetLoaders)
from DatasetLoaders import HRSIDSegmentationDataset
import MaskRCNN_model_wrapper
importlib.reload(MaskRCNN_model_wrapper)
# from MaskRCNN_model_wrapper import MaskRCNNWrapper
from MaskRCNN_model_wrapper import do_training, save_model, get_model
import fiftyone_evaluation as fiftyone_evaluation



In [2]:
# get path to directory above
p = Path()
top_dir = p.absolute().parents[1]
top_dir

PosixPath('/home/ENEE439D_SAR_Ship/SAR-ShipDetection')

In [3]:
name = "HRSID_train"
if name in fo.list_datasets():
    dataset_train = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "train2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_train = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

 100% |███████████████| 3642/3642 [24.6s elapsed, 0s remaining, 124.1 samples/s]      


In [4]:
name = "HRSID_test"
if name in fo.list_datasets():
    dataset_test = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "test2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_test = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

 100% |███████████████| 1961/1961 [12.6s elapsed, 0s remaining, 102.2 samples/s]      


In [5]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

In [6]:
session = fo.launch_app()

In [7]:
def get_transform_lee():
    transforms = []

    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    transforms.append(T.LeeFilter(5))
    
    return T.Compose(transforms)

In [8]:
model = get_model(2, True)

In [9]:
HRSID_torch_train_lee = HRSIDSegmentationDataset(dataset_train, transforms=get_transform_lee())
HRSID_torch_test_lee = HRSIDSegmentationDataset(dataset_test, transforms=get_transform_lee())

In [ ]:
do_training(model, HRSID_torch_train_lee, HRSID_torch_test_lee, num_epochs=45, step_size=15)

In [ ]:
save_model(model, "maskrcnn_hrsid_pre_lee_e40_lr005_m5_step15.pt")

In [ ]:
fiftyone_evaluation.add_detections(model, HRSID_torch_test_lee, dataset_test, field_name="predictions_lee")

In [ ]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_lee", 
    classes=["ship"],
    eval_key="eval", 
    compute_mAP=True,
    gt_field='detections'
)

In [ ]:
results.mAP()

In [ ]:
results.print_report()

In [7]:
def get_transform_HF_jitter():
    transforms = []

    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    transforms.append(T.RandomHorizontalFlip())
    transforms.append(T.ScaleJitter((600, 600)))
    
    return T.Compose(transforms)

In [12]:
model = get_model(2, True)

In [13]:
HRSID_torch_train_HF_jitter = HRSIDSegmentationDataset(dataset_train, transforms=get_transform_HF_jitter())
HRSID_torch_test_HF_jitter = HRSIDSegmentationDataset(dataset_test, transforms=get_transform_HF_jitter())

In [14]:
do_training(model, HRSID_torch_train_HF_jitter, HRSID_torch_test_HF_jitter, num_epochs=45, step_size=15)

Using cuda


/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/utils/tensor_new.cpp:245.)
  masks=torch.as_tensor(masks, dtype=torch.uint8)
/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/utils/tensor_new.cpp:245.)
  masks=torch.as_tensor(masks, dtype=torch.uint8)
/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list

Epoch: [0]  [  0/456]  eta: 3:02:20  lr: 0.000016  loss: 3.5779 (3.5779)  loss_classifier: 0.9416 (0.9416)  loss_box_reg: 0.1393 (0.1393)  loss_mask: 1.5114 (1.5114)  loss_objectness: 0.8353 (0.8353)  loss_rpn_box_reg: 0.1502 (0.1502)  time: 23.9921  data: 23.3733  max mem: 6503
Epoch: [0]  [ 10/456]  eta: 0:22:41  lr: 0.000126  loss: 2.9146 (3.1898)  loss_classifier: 0.9007 (0.8440)  loss_box_reg: 0.1393 (0.1352)  loss_mask: 1.5114 (1.4710)  loss_objectness: 0.1361 (0.6564)  loss_rpn_box_reg: 0.0236 (0.0831)  time: 3.0527  data: 2.4746  max mem: 6648
Epoch: [0]  [ 20/456]  eta: 0:18:48  lr: 0.000236  loss: 1.9813 (2.5127)  loss_classifier: 0.5710 (0.6442)  loss_box_reg: 0.1565 (0.1489)  loss_mask: 1.0491 (1.1679)  loss_objectness: 0.0814 (0.4889)  loss_rpn_box_reg: 0.0140 (0.0628)  time: 1.5188  data: 0.9421  max mem: 6648
Epoch: [0]  [ 30/456]  eta: 0:14:12  lr: 0.000345  loss: 1.2492 (2.0494)  loss_classifier: 0.2924 (0.5123)  loss_box_reg: 0.1649 (0.1542)  loss_mask: 0.6002 (0.9528

/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/utils/tensor_new.cpp:245.)
  masks=torch.as_tensor(masks, dtype=torch.uint8)
/home/ENEE439D_SAR_Ship/miniconda3/envs/torch-310-cuda11.7/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on T

creating index...
index created!
Test:  [  0/491]  eta: 0:53:42  model_time: 0.3774 (0.3774)  evaluator_time: 0.2376 (0.2376)  time: 6.5622  data: 5.9323  max mem: 8442
Test:  [100/491]  eta: 0:05:13  model_time: 0.1538 (0.1894)  evaluator_time: 0.0393 (0.0693)  time: 0.7961  data: 0.4888  max mem: 8442
Test:  [200/491]  eta: 0:03:51  model_time: 0.1605 (0.1937)  evaluator_time: 0.0425 (0.0731)  time: 0.5621  data: 0.3183  max mem: 8442
Test:  [300/491]  eta: 0:02:27  model_time: 0.1522 (0.1865)  evaluator_time: 0.0504 (0.0673)  time: 0.8991  data: 0.6697  max mem: 8442
Test:  [400/491]  eta: 0:01:21  model_time: 0.2470 (0.2253)  evaluator_time: 0.1189 (0.1034)  time: 1.7471  data: 1.1632  max mem: 8442
Test:  [490/491]  eta: 0:00:01  model_time: 0.2081 (0.2565)  evaluator_time: 0.0843 (0.1344)  time: 0.4943  data: 0.0140  max mem: 8442
Test: Total time: 0:08:55 (1.0913 s / it)
Averaged stats: model_time: 0.2081 (0.2565)  evaluator_time: 0.0843 (0.1344)
Accumulating evaluation results.

KeyboardInterrupt: 

In [15]:
save_model(model, "maskrcnn_hrsid_pre_HF_jitter_e45_lr005_m5_step15.pt")

In [ ]:
fiftyone_evaluation.add_detections(model, HRSID_torch_test_HF_jitter, dataset_test, field_name="predictions_HF_jitter")

Using device cuda
   0% ||--------------|    0/1961 [218.6ms elapsed, ? remaining, ? samples/s] 

/home/ENEE439D_SAR_Ship/miniconda3/envs/torch-310-cuda11.7/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


   1% ||--------------|   16/1961 [8.6s elapsed, 17.5m remaining, 2.0 samples/s] 

In [19]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_HF_jitter", 
    classes=["ship"],
    eval_key="eval", 
    compute_mAP=True,
    gt_field='detections'
)

Evaluating detections...
 100% |███████████████| 1961/1961 [1.7m elapsed, 0s remaining, 19.5 samples/s]       
Performing IoU sweep...


 100% |███████████████| 1961/1961 [47.3s elapsed, 0s remaining, 56.2 samples/s]      


In [ ]:
results.mAP()

In [ ]:
results.print_report()